In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval as make_tuple
import os

### Read all relevant data files 🦋

In [2]:
days = ["p2", "p3", "p4", "p5", "p6", "p7"]
filepath = "validation_data/feature_extraction/feature/"
alldata = [pd.read_excel("{}{}_alldata.xlsx".format(filepath,x), 
                         sheet_name="Sheet1", engine='openpyxl') for x in days]
degree = [pd.read_excel("{}{}_degreedata.xlsx".format(filepath,x), 
                        sheet_name="Sheet1", engine='openpyxl') for x in days]

### Generate np arrays
Returns list of centered np arrays, where each array corresponds to a day.

In [3]:
arrays_by_day = []
for df in degree:
    to_array = [make_tuple(x) for x in df["nodes"]]
    coords = np.array(to_array)
    coords = coords - np.mean(coords, axis=0)
    arrays_by_day.append(coords)

In [4]:
# Verify that the data is centered
for a in arrays_by_day:
    print(sum(a).round(6))

[-0.  0.]
[ 0. -0.]
[0. 0.]
[-0. -0.]
[-0. -0.]
[0. 0.]


### Generate edge list with weights 🐙
Output: 'i j weight' where $i,j\in \mathbb{Z}$

In [8]:
# map nodes to integers using row index of degreedata.xlsx
weight = "width" # try "width" or "length"
edges = []
for i in range(len(alldata)):
    node_list = list(degree[i]["nodes"])
    node1 = [node_list.index(x)+1 for x in alldata[i]["node1"]]
    node2 = [node_list.index(x)+1 for x in alldata[i]["node2"]]
    df = pd.DataFrame([node1,node2,alldata[i][weight]])
    df = df.astype(str)
    df = df.replace(to_replace = "\.0+$",value = "", regex = True)
    edges.append(df.T)

In [9]:
# writes to txt files in new folder "matlab_input"
if not os.path.exists('./matlab_input'):
    os.mkdir('./matlab_input')
for i in range(len(edges)):
    edges[i].to_csv(r'matlab_input/{}_weighted.txt'.format(days[i]),index=False,sep=" ",header=False)